## Overview
In this notebook, we will demonstrate that the outputs of the Julia DCON conversion are equivalent to those in the Fortran version by comparing the eigenmodes of the DIIID ideal example

In [1]:
# Load in necessary packages
using Pkg
using FortranFiles
using LinearAlgebra
using HDF5
using Plots
using LaTeXStrings

## Load in Fortran data

For convenience, we have saved the Fortran `euler.bin` file here for comparison. Below, we write a Julia version of the logic within `idcon.f` in GPEC, which reads in the `euler.bin` data and converts it to usable form

In [2]:
# Mirroring gpec solution structs from fortran idcon.f
# Stores solutions read directly from output file
struct solution_type
    msol :: Int32
    u :: Array{ComplexF64,3}
end

# Stores information for resistive calculations
struct resist_type
    e :: Float64
    f :: Float64
    h :: Float64
    m :: Float64
    g :: Float64
    k :: Float64
    eta :: Float64
    rho :: Float64
    taua :: Float64
    taur :: Float64
    di :: Float64
    dr :: Float64
    sfac :: Float64
    deltac :: Float64
    eigenvalue :: ComplexF64
end

# Stores information about singular surfaces
struct sing_type
    msol_l :: Int32
    msol_r :: Int32
    jfix :: Int32
    jpert :: Int32
    psifac :: Float64
    q :: Float64
    q1 :: Float64
    ca_l :: Array{ComplexF64,3}
    ca_r :: Array{ComplexF64,3}
    restype :: resist_type
end

# Stores fixfacs
mutable struct fixfac_type
    msol :: Int32
    index :: Vector{Int32}
    fixfac :: Array{ComplexF64,2}
    transform :: Array{ComplexF64,2}
    gauss :: Array{ComplexF64,2}
    function fixfac_type(msol::Int32, index::Vector{Int32},fixfac::Array{ComplexF64,2})
        transform = Matrix{ComplexF64}(I,msol,msol)
        gauss = Matrix{ComplexF64}(I,msol,msol)
        new(msol,index,fixfac,transform,gauss)
    end
end

# Sets up fixfacs to recover true eigenmodes - Julia conversion of the idcon_transform subroutine in GPEC
function idcon_transform(fixtype, sing_flag)
    
    msol = length(fixtype[2].index)
    mfix = length(fixtype)-1
    identity = Matrix{ComplexF64}(I,msol,msol)

    # Construct gaussian reduction matrix
    for ifix in 2:1:mfix+1
        fixtype[ifix].gauss = copy(identity)
        mask = zeros(Bool, msol)
        mask .= true
        for isol in 1:msol
            ksol = fixtype[ifix].index[isol]
            mask[ksol] = false
            temp = copy(identity)
            for jsol in 1:msol
                if mask[jsol]
                    temp[ksol,jsol] = fixtype[ifix].fixfac[ksol,jsol]
                end
            end
            # Matrix multiplication gauss = gauss * temp
            fixtype[ifix].gauss = fixtype[ifix].gauss * temp
        end
        if sing_flag[ifix-1]
            fixtype[ifix].gauss[:,fixtype[ifix].index[1]] .= 0.0
        end
    end
    
    # Concatenate gaussian reduction matrix
    fixtype[mfix+1].transform = copy(identity)
    for ifix in mfix:-1:1
        fixtype[ifix].transform = fixtype[ifix+1].gauss * fixtype[ifix+1].transform
    end
    return fixtype
end

idcon_transform (generic function with 1 method)

In [3]:
# Euler.bin file from gpec develop branch for comparison
fstream = FortranFile("euler.bin", "r")

# --- Read header ---
mlow, mhigh, nn, mpsi, mtheta, ro, zo = read(fstream, Int32, Int32, Int32, Int32, Int32, Float64, Float64)
mband, mthsurf0, mthvac, psio, psilow, psilim, qlim, singfac_min = read(fstream, Int32, Float64, Int32, Float64, Float64, Float64, Float64, Float64)
power_b, power_r, power_bp = read(fstream, Int32, Int32, Int32)
kin_flag, con_flag = read(fstream, Int32, Int32)

amean, rmean, aratio, kappa, delta1, delta2, 
    li1, li2, li3, betap1, betap2, betap3, betat, betan, bt0, 
    q0, qmin, qmax, qa, crnt, q95, shotnum, shottime = read(fstream, 
        Float64, Float64, Float64, Float64, Float64, Float64,
        Float64, Float64, Float64, Float64, Float64, Float64,
        Float64, Float64, Float64, Float64, Float64, Float64,
        Float64, Float64, Float64, Float64, Float64)

# --- Read equilibrium on flux coordinates ---
# sq%xs is size mpsi, sq%fs is size mpsi x 4, sq%fs1 is size mpsi x 4, sq%xpower is size 2 x 4
sq_xs, sq_fs, sq_fs1, sq_xpower = read(fstream, (Float64, mpsi), (Float64, (mpsi,4)), (Float64, (mpsi,4)), (Float64, (2,4)))
# rzphi%xs is size mpsi, rzphi%ys is size mtheta, rzphi%fs is size mpsi x mtheta, etc.
rzphi_xs, rzphi_ys, 
    rzphi_fs, rzphi_fsx, rzphi_fsy, rzphi_fsxy, 
    rzphi_x0, rzphi_y0, rzphi_xpower, rzphi_ypower = read(fstream, 
        (Float64, mpsi), (Float64, mtheta), 
        (Float64, (mpsi,mtheta,4)), (Float64, (mpsi,mtheta,4)), (Float64, (mpsi,mtheta,4)), (Float64, (mpsi,mtheta,4)),
        (Float64, 2), (Float64, 2), (Float64, (2,4)), (Float64, (2,4)))

# misc derived quantities
chi1=2π*psio
mpert=mhigh-mlow+1

# --- Count solutions in file (first pass) ---
mstep = -1
mfix = 0
msing = 0

while !eof(fstream)
    data_type = read(fstream, Int32)
    if data_type == 1
        mstep += 1
        read(fstream)  # skip
        read(fstream)
        read(fstream)
    elseif data_type == 2
        mfix += 1
        read(fstream)
        read(fstream)
    elseif data_type == 3
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
    elseif data_type == 4
        msing += 1
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
    elseif data_type == 5
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
        read(fstream)
    else
        error("Cannot recognize data_type = $data_type")
    end
end
close(fstream)

# --- Allocate arrays ---
psifac = zeros(Float64, mstep+1)
rhofac = zeros(Float64, mstep+1)
qfac = zeros(Float64, mstep+1)

soltype = Vector{solution_type}(undef, mstep+1)
singtype = Vector{sing_type}(undef, msing)
fixstep = zeros(Int32, mfix+2)
sing_flag = zeros(Bool, mfix)
fixtype = Vector{fixfac_type}(undef, mfix+1)

# Initialize first fixfac
fixtype[1] = fixfac_type(Int32(0), zeros(Int32, 0), zeros(ComplexF64, 0, 0))

et = zeros(ComplexF64, mpert)
ep = zeros(ComplexF64, mpert)
ee = zeros(ComplexF64, mpert)
wt = zeros(ComplexF64, (mpert,mpert))
wt0 = zeros(ComplexF64, (mpert,mpert))
wft = zeros(ComplexF64, (mpert,mpert))
eft = zeros(ComplexF64, mpert)
efp = zeros(ComplexF64, mpert)
wtraw = zeros(ComplexF64, (mpert,mpert))

eft .= -1
wft .= 0
fixstep[1] = 0
fixstep[mfix+2] = mstep

fstream = FortranFile("euler.bin", "r")
# --- Read header again ---
mlow, mhigh, nn, mpsi, mtheta, ro, zo = read(fstream, Int32, Int32, Int32, Int32, Int32, Float64, Float64)
mband, mthsurf0, mthvac, psio, psilow, psilim, qlim, singfac_min = read(fstream, Int32, Float64, Int32, Float64, Float64, Float64, Float64, Float64)
power_b, power_r, power_bp = read(fstream, Int32, Int32, Int32)
kin_flag, con_flag = read(fstream, Int32, Int32)
istep = -1
ifix = 0
ising = 0

while !eof(fstream)
    data_type = read(fstream, Int32)
    if data_type == 1
        istep += 1
        psifac[istep+1], qfac[istep+1], msoli = read(fstream, Float64, Float64, Int32)
        u = Array{ComplexF64}(undef,(mpert,msoli,4))
        u[:,:,1:2] = read(fstream,  (ComplexF64,(mpert,msoli,2)))
        u[:,:,3:4] = read(fstream,  (ComplexF64,(mpert,msoli,2)))
        soltype[istep+1] = solution_type(msoli,u)
    elseif data_type == 2
        ifix += 1
        fixstep[ifix+1] = istep
        sing_flag[ifix], msoli = read(fstream, Int32, Int32)
        fixfac, index = read(fstream, (ComplexF64,(msoli,msoli)),(Int32,msoli))
        fixtype[ifix+1] = fixfac_type(msoli,index,fixfac)
    elseif data_type == 3
        ep = read(fstream,(ComplexF64,mpert))
        et = read(fstream,(ComplexF64,mpert))
        wt = read(fstream,(ComplexF64,(mpert,mpert)))
        wt0 = read(fstream,(ComplexF64,(mpert,mpert)))
        wv_farwall_flag = read(fstream, Int32)
    elseif data_type == 4
        ising += 1
        jfixi = ifix
        jperti = round(Int32, nn*qfac[istep+1] + 0.5)
        psifaci, qi, q1i = read(fstream, Float64, Float64, Float64)
        msol_l = read(fstream, Int32)
        ca_l = read(fstream, (ComplexF64,(mpert,msol_l,2)))
        msol_r = read(fstream, Int32)
        ca_r = read(fstream, (ComplexF64,(mpert,msol_r,2)))
        e, f, h, m, g, k, eta, rho, taua, taur = read(fstream, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64)
    elseif data_type == 5
        ep = read(fstream,(ComplexF64,mpert))
        et = read(fstream,(ComplexF64,mpert))
        wt = read(fstream,(ComplexF64,(mpert,mpert)))
        efp = read(fstream,(Float64,mpert))
        eft = read(fstream,(Float64,mpert))
        wft = read(fstream,(ComplexF64,(mpert,mpert)))
        wtraw = read(fstream,(ComplexF64,(mpert,mpert)))
    else
        # for all the reads we skipped in the second pass
        print()
    end
end
fixstep .+= 1 # Fortran indexes from 0 for steps, adjust
close(fstream)

# Set up fortran fixfacs
fixfacproc = idcon_transform(fixtype, sing_flag)

println("Done reading euler.bin")

Done reading euler.bin


## Load in Julia data
We will run the main DCON code on the same inputs. We output the `euler.h5` file, which is a Julia version of the `euler.bin` file.

In [4]:
# Run DCON in Julia
Pkg.activate("../..")
using JPEC
JPEC.DCON.Main("./") # "./" tells us to obtain inputs and direct outputs to our current folder

  Activating project at `~/Github/JPEC`


DCON START
----------------------------------


┌ Info: Forcing hamada coordinate jacobian exponents: power_*
└ @ JPEC.Equilibrium /Users/jakehalpern/Github/JPEC/src/Equilibrium/EquilibriumTypes.jl:48


Equilibrium file: g147131.02300_DIIID_KEFIT
--> Processing EFIT g-file: g147131.02300_DIIID_KEFIT
--> Parsed from header: nw=129, nh=129
--> All main data blocks parsed successfully.
--> Creating 1D profile splines...
--> 1D Spline fitting complete.
--> Creating 2D psi spline...
--> 2D Spline fitting complete.
--- Starting Direct Equilibrium Processing ---
Finding magnetic axis...
  Iter  1: R = 1.760526, Z = 0.025705, |ΔR|=7.05e-02, |ΔZ|=2.57e-02
  Iter  2: R = 1.758881, Z = 0.024420, |ΔR|=1.64e-03, |ΔZ|=1.29e-03
  Iter  3: R = 1.758880, Z = 0.024419, |ΔR|=6.85e-07, |ΔZ|=8.76e-07
  Iter  4: R = 1.758880, Z = 0.024419, |ΔR|=2.58e-13, |ΔZ|=2.96e-13
Magnetic axis found at R=1.7588803190504143, Z=0.024419144244256175.
Finding inboard separatrix crossing...
  Restart attempt 1/6 with initial R = 1.529160
inboard separatrix found at R=1.0872289007986813.
Finding outboard separatrix crossing...
  Restart attempt 1/6 with initial R = 1.954160
outboard separatrix found at R=2.292018620764088.


In [ ]:
# Read in from julia hdf5 for comparison
eh5 = h5open("euler.h5", "r")
mstepj = read(eh5["integration/nstep"])
xi_psij = read(eh5["integration/xi_psi"])
psifacj = read(eh5["integration/psi"])
psioj = read(eh5["equil/psio"])
wtj = read(eh5["vacuum/wt"])
close(eh5)

# scale energy eigenvector matrices
chi1j = 2π*psioj
wtf = wt*(chi1*1e-3)
wtj = wtj*(chi1j*1e-3)
println("Done reading euler.h5")

# Compare Outputs
We will now perform the equivalent `idcon_build`, which builds the ideal Euler-Langrange solutions for a given eigenmode. We have performed the transformation and scaling necessary to obtain the physical quantities ($\Xi_\Psi, \Xi_s$ etc. ) typically calculated in GPEC preprocessing in `idcon.f` and saved them in the HDF5 file. Notice how this makes the Julia outputs postprocessing much simpler, as we don't need to call `idcon_transform` or `idcon_build` in this notebook.

In [ ]:
xi_psif = zeros(ComplexF64, (mstep+1,mpert))

# Construct uedge coefficients using the most unstable mode (index 1)
uedgef = soltype[length(soltype)].u[:,:,1] \ wtf[:,1]

# Create Fortran eigenfunctions
jfix = 1
for ifix in 1:mfix+1
    temp1 = fixtype[ifix].transform * uedgef
    kfix = fixstep[ifix+1]
    for istep in jfix:kfix
        xi_psif[istep,:] = soltype[istep].u[:,:,1] * temp1
    end
    jfix = kfix + 1
end

## Plots

### Plot comparison of xi_psi for a few poloidal mode numbers

In [ ]:
p = plot()
# In the Julia output, xi_psij is indexed as (mpert, nmode, nstep), so we extract nmode=1 for the least stable mode
for m in 1:4
    plot!(psifac, imag.(xi_psif[:,m - mlow + 1]), label="Fortran, m=$m")
    plot!(psifacj, imag.(xi_psij[m - mlow + 1, 1, :]), linestyle=:dash, label="Julia, m=$m")
end
xlabel!(L"\psi_N")
ylabel!(L"\mathrm{Im}(\xi_\psi)")
title!("Comparison of " * L"\xi_\psi" * " for " * "m=1-4")
display(p)

### Plot comparison of psifac integration
We can see that in this scenario, the Julia integrator is actually more efficient than LSODE in Fortran. One reason is because we can see LSODE is extremely jumpy around solution normalizations, and slows down directly after these regions.

In [ ]:
p = plot(psifac, label="Fortran")
# plot vertical line at each fixstep
for ifix in 2:1:mfix+1
    vline!([fixstep[ifix]+1], linestyle=:dash, color=:black, label="", linewidth=0.25, alpha=0.5)
end
plot!(psifacj, linestyle=:dash, label="Julia")
xaxis!("ODE step index")
yaxis!(L"\psi_N")
display(p)

### Compare the eigenvectors of each DCON energy matrix eigenmode

In [ ]:
l = @layout [a{0.97w} b{0.03w}]

# Determine the global minimum and maximum values for the color scale
global_min = minimum([minimum(log10.(abs.(wtf'))), minimum(log10.(abs.(wtj'))), minimum(log10.(abs.((wtf'-wtj'))))])
global_max = maximum([maximum(log10.(abs.(wtf'))), maximum(log10.(abs.(wtj'))), maximum(log10.(abs.((wtf'-wtj'))))])

# Common axis labels
xlabel = "m"
ylabel = "mode"

# Create the heatmaps without individual colorbars
p1 = heatmap(
    log10.(abs.(wtf')), 
    aspect_ratio=1, 
    title="Fortran " * L"log_{10} |W_t|", 
    xlabel=xlabel, ylabel=ylabel,
    size=(400,400), interpolate=false, fxaa=false, 
    color=:viridis, clims=(global_min, global_max), colorbar=false, left_margin=5Plots.mm, bottom_margin=5Plots.mm, top_margin=5Plots.mm, right_margin=2Plots.mm
)

p2 = heatmap(
    log10.(abs.(wtj')), 
    aspect_ratio=1, 
    title="Julia " * L"log_{10} |W_t|", 
    xlabel=xlabel, ylabel=ylabel,
    size=(400,400), interpolate=false, fxaa=false, 
    color=:viridis, clims=(global_min, global_max), colorbar=false, left_margin=5Plots.mm, bottom_margin=5Plots.mm, top_margin=5Plots.mm, right_margin=2Plots.mm
)

p3 = heatmap(
    log10.(abs.((wtf'-wtj'))), 
    aspect_ratio=1, 
    title=L"log_{10} \left|W_{tf} - W_{tj}\right|", 
    xlabel=xlabel, ylabel=ylabel,
    size=(400,400), interpolate=false, fxaa=false, 
    color=:viridis, clims=(global_min, global_max), colorbar=false, left_margin=5Plots.mm, bottom_margin=5Plots.mm, top_margin=5Plots.mm, right_margin=2Plots.mm
)

# Add a single color bar to the right of the combined plots
cb = heatmap(
    zeros(2,2), 
    color=:viridis, clims=(global_min, global_max), 
    colorbar=true, framestyle=:none, 
    xaxis=false, yaxis=false, 
    size=(40,400), right_margin=5Plots.mm, left_margin=0Plots.mm, 
    lims=(-1,0)
)

# Combine the plots with a single shared color bar on the right
p = plot(p1, p2, p3, layout=(1,3), size=(1200,400), margin=0Plots.mm)
pp = plot(p, cb, layout=l, size=(1300,400), margin=0Plots.mm)
display(pp)